In [1]:
import numpy as np
import pandas as pd
import datetime
import time
from model import Net1
import torch.optim as optim
from scipy import io
import argparse
import os                    # nn.BatchNorm2d(2,affine=False),
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import h5py  
import matplotlib.pyplot as plt
import h5py  
import matplotlib
from PIL import Image
import math
from sklearn.metrics import confusion_matrix
import pylab as pl
import matplotlib.pyplot as plt
import numpy as np
import itertools
import ParaSetting


In [3]:

torch.cuda.synchronize()
starttime = time.time()

os.environ["CUDA_VISIBLE_DEVICES"]="0" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
read_h5 = True
num_workers = 0


## Working path define
expname  = ParaSetting.expname
root_ws   = ParaSetting.root_ws

epoch_num_char = str(ParaSetting.epoch_num)

datapath  = 'D:/Ultra-low-field-MRI-Scanner-main/hku__EMI/dataset1/'+'data/'
modelpath_test = 'D:/Ultra-low-field-MRI-Scanner-main/hku__EMI/dataset1/'+'model/'+'epoch-'+epoch_num_char+'.pth'
savepath  = 'D:/Ultra-low-field-MRI-Scanner-main/hku__EMI/dataset1/'+'results/'
os.makedirs(savepath, exist_ok=True)

## Hyperparameters
Nx = ParaSetting.Nx
bs = ParaSetting.bs  # batch size


cuda


In [6]:



class prepareData_test(Dataset):
    def __init__(self, train_or_test):

       self.files = os.listdir(datapath+train_or_test)
       self.files.sort(key=lambda x:int(x[:-4])) 
       self.train_or_test= train_or_test

    def __len__(self):
       return len(self.files)

    def __getitem__(self, idx):
        
        data = torch.load(datapath+self.train_or_test+'/'+self.files[idx])
        return data['k-space'],  data['label']
   

testset = prepareData_test('test')
testloader = torch.utils.data.DataLoader(testset, batch_size=1,shuffle=False, num_workers=num_workers)


# Testing
filename = os.listdir('./dataset1/data/test')
length = len(filename)

din = np.empty((length,2,Nx,10))
dout = np.empty((length,2,Nx,1))
dlab = np.empty((length,2,Nx,1))
print(modelpath_test)

model = torch.load(modelpath_test)

criterion1 = nn.MSELoss()


D:/Ultra-low-field-MRI-Scanner-main/hku__EMI/dataset1/model/epoch-20.pth


In [7]:
model.eval()  # 切换到评估模式
loss_validation_list = []
dout = []
dlab = []

print('\nTesting...')

# 遍历测试集
for i, data in enumerate(testloader, 0):
    inputs = data[0].reshape(-1, 2, Nx, 10).to(device)  # 输入 reshape 并移动到设备
    labels = data[1].reshape(-1, 2, Nx, 1).to(device)  # 标签 reshape 并移动到设备

    # 不计算梯度，进入推理模式
    with torch.no_grad():
        outs = model(inputs)  # 前向传播，得到输出

    # 计算损失
    loss = criterion1(outs, labels)
    loss_validation_list.append(round(loss.item(), 10))  # 记录损失值并保留 10 位小数

    # 打印当前损失列表
    print(f'Batch {i+1}/{len(testloader)}, Loss: {loss_validation_list[-1]}')

    # 将输出、标签和输入从 GPU 转移到 CPU，并转换为 numpy 数组
    output = outs.cpu().numpy()
    labelo = labels.cpu().numpy()

    # 存储输出和标签
    dout.append(output[0:1, :, :, :])
    dlab.append(labelo[0:1, :, :, :])

# 计算整个测试集的平均损失
average_loss = sum(loss_validation_list) / len(loss_validation_list)
print(f'\nAverage Validation Loss: {average_loss}')

# 返回损失列表和存储的输出、标签



Testing...
Batch 1/8064, Loss: 2.5929582119
Batch 2/8064, Loss: 2.3125224113
Batch 3/8064, Loss: 2.0331053734
Batch 4/8064, Loss: 2.5410339832
Batch 5/8064, Loss: 2.6406989098
Batch 6/8064, Loss: 2.8998584747
Batch 7/8064, Loss: 2.6841583252
Batch 8/8064, Loss: 2.5396769047
Batch 9/8064, Loss: 2.2660226822
Batch 10/8064, Loss: 2.2501683235
Batch 11/8064, Loss: 2.5448694229
Batch 12/8064, Loss: 2.0800790787
Batch 13/8064, Loss: 2.4584472179
Batch 14/8064, Loss: 2.1116931438
Batch 15/8064, Loss: 2.3475925922
Batch 16/8064, Loss: 2.2372655869
Batch 17/8064, Loss: 2.25563097
Batch 18/8064, Loss: 2.7562634945
Batch 19/8064, Loss: 2.0813932419
Batch 20/8064, Loss: 2.2737782001
Batch 21/8064, Loss: 2.4762320518
Batch 22/8064, Loss: 2.0625710487
Batch 23/8064, Loss: 2.2553579807
Batch 24/8064, Loss: 2.472155571
Batch 25/8064, Loss: 2.4712138176
Batch 26/8064, Loss: 2.4050171375
Batch 27/8064, Loss: 2.5323591232
Batch 28/8064, Loss: 2.322406292
Batch 29/8064, Loss: 2.5081710815
Batch 30/8064, 

In [8]:
print(max(loss_validation_list))

131455.90625


In [5]:

f = h5py.File(savepath+'output.h5','w')
f['k-space'] = dout
k = h5py.File(savepath+'label.h5','w')
k['k-space'] = dlab
f.close()
k.close()
